#### Import libraries

In [2]:
import pandas as pd
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt
from pandas import DataFrame as df
from sklearn import preprocessing


#### Import dataset

In [3]:
#import Data File to CSV
#import Data File to CSV
export30 = pd.read_csv('export30_drop_duplicated.csv')
export31 = pd.read_csv('export31_drop_duplicated.csv')
export32 = pd.read_csv('export32_drop_duplicated.csv')
export33 = pd.read_csv('export33_drop_duplicated.csv')
frames = [export30, export31, export32,export33]
state = pd.concat(frames).drop_duplicates()

state.groupby('PRTNR_ID').value_counts()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_counts'

#### *Delete the employee whose active status is always 0

In [2456]:
#Delete the employee whose active status is always 0
# Find employees whose active status is alwyas 0
state1 = pd.DataFrame(state.groupby(['PRTNR_ID'])['PRTNR_ACTIVE_FLG'].sum())
state1 = state1['PRTNR_ACTIVE_FLG'].reset_index()
state1 = state1[state1['PRTNR_ACTIVE_FLG'] == 0]
state1 = state1.drop(['PRTNR_ACTIVE_FLG'], axis=1).reset_index().drop(['index'], axis=1)
state1 =list(state1['PRTNR_ID'])

In [2457]:
#drop the partners row whose active stats is always 0
to_drop = state1
state = state[~state['PRTNR_ID'].isin(to_drop)]

#### *change time format

In [2458]:
state['FiscalWeekBeginDate']=pd.to_datetime(state['FiscalWeekBeginDate'])

#### count employee

In [2459]:
# Total # of Employees
len(state['PRTNR_ID'].unique())

13491

#### * get a dataframe contains Unique Partner's ID 

In [2460]:
unique_prtnr_ID = pd.DataFrame(state['PRTNR_ID'].unique())
unique_prtnr_ID = unique_prtnr_ID.rename(columns = {0:'PRTNR_ID'})

#### * Each Employee's the earliest start date (=FiscalWeekBeginDate min_value)

In [2461]:
earliest_FiscalWeekBeginDate = state.groupby(['PRTNR_ID'])['FiscalWeekBeginDate'].min()
earliest_FiscalWeekBeginDate = earliest_FiscalWeekBeginDate.reset_index()

#### * The number of positions that employees covered

In [2462]:
# each employees' job count
employee_jobs = state[['PRTNR_ID','JOB_NM']].drop_duplicates()
the_number_of_jobs = employee_jobs.groupby(['PRTNR_ID'])['JOB_NM'].count().reset_index()
the_number_of_jobs = the_number_of_jobs.rename(columns={'JOB_NM': 'count_position'})

#### * The last position name

In [2463]:
# Find the last job position name
last_position = state[['PRTNR_ID','FiscalWeekBeginDate', 'JOB_NM']]
last_position = last_position.groupby('PRTNR_ID').max().reset_index().drop(['FiscalWeekBeginDate'], axis=1)
last_position = last_position.rename(columns = {'JOB_NM':'last_position'})

#### * Obtaining the last Com_Tenure,   last FiscalWeekBeginDate and the last status of PRTNR_ACTIVE_FLG

In [2464]:
# Collect the cols 'PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_JOB_TNUR' from the original dataset
FSCL_WK_END_DT_COM_TNUR = state[['PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_COM_TNUR','PRTNR_ACTIVE_FLG']]

#Employee whose last active flg = 0
FSCL_WK_END_DT_COM_TNUR_0 = FSCL_WK_END_DT_COM_TNUR[FSCL_WK_END_DT_COM_TNUR['PRTNR_ACTIVE_FLG'] == 0]
FSCL_WK_END_DT_COM_TNUR_0 =  FSCL_WK_END_DT_COM_TNUR_0.groupby(['PRTNR_ID','FSCL_WK_END_DT_COM_TNUR','PRTNR_ACTIVE_FLG'])['FiscalWeekBeginDate'].agg('min')
FSCL_WK_END_DT_COM_TNUR_0 = pd.DataFrame(FSCL_WK_END_DT_COM_TNUR_0).reset_index()

In [2465]:
#Employee whose last active flg = 1
FSCL_WK_END_DT_COM_TNUR = state[['PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_COM_TNUR','PRTNR_ACTIVE_FLG']]
FSCL_WK_END_DT_COM_TNUR_1 = FSCL_WK_END_DT_COM_TNUR[FSCL_WK_END_DT_COM_TNUR['PRTNR_ACTIVE_FLG'] == 1]
FSCL_WK_END_DT_COM_TNUR_1 =  FSCL_WK_END_DT_COM_TNUR_1.groupby(['PRTNR_ID'])['FSCL_WK_END_DT_COM_TNUR','FiscalWeekBeginDate','PRTNR_ACTIVE_FLG'].agg('max')
FSCL_WK_END_DT_COM_TNUR_1 = pd.DataFrame(FSCL_WK_END_DT_COM_TNUR_1).reset_index()
# #drop the partners row whose active stats is always 0
to_drop = FSCL_WK_END_DT_COM_TNUR_0['PRTNR_ID']
FSCL_WK_END_DT_COM_TNUR_1['PRTNR_ID'] = FSCL_WK_END_DT_COM_TNUR_1[~FSCL_WK_END_DT_COM_TNUR_1['PRTNR_ID'].isin(to_drop)]
FSCL_WK_END_DT_COM_TNUR_1=FSCL_WK_END_DT_COM_TNUR_1.dropna()
FSCL_WK_END_DT_COM_TNUR_1

#concat FSCL_WK_END_DT_COM_TNUR_0 and FSCL_WK_END_DT_COM_TNUR_1
FSCL_WK_END_DT_COM_TNUR =pd.concat([FSCL_WK_END_DT_COM_TNUR_0,FSCL_WK_END_DT_COM_TNUR_1], axis=0)
FSCL_WK_END_DT_COM_TNUR['Last_Month_Year'] = pd.to_datetime(FSCL_WK_END_DT_COM_TNUR['FiscalWeekBeginDate']).dt.to_period('M')
FSCL_WK_END_DT_COM_TNUR = FSCL_WK_END_DT_COM_TNUR.rename(columns = {'FiscalWeekBeginDate' : 'Last_FiscalWeekBeginDate',
                                                                   'FSCL_WK_END_DT_COM_TNUR': 'Last_FSCL_WK_END_DT_COM_TNUR'})

<ipython-input-2465-6a6bac8a7aaf>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  FSCL_WK_END_DT_COM_TNUR_1 =  FSCL_WK_END_DT_COM_TNUR_1.groupby(['PRTNR_ID'])['FSCL_WK_END_DT_COM_TNUR','FiscalWeekBeginDate','PRTNR_ACTIVE_FLG'].agg('max')


#### *the longest position holding period

In [2466]:
the_longest_position_holding_period = state[['PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_JOB_TNUR']]
the_longest_position_holding_period = pd.merge(FSCL_WK_END_DT_COM_TNUR,the_longest_position_holding_period, how ='inner',
                                              left_on = ['PRTNR_ID','Last_FiscalWeekBeginDate'],
                                                         right_on = ['PRTNR_ID','FiscalWeekBeginDate']).drop_duplicates()

the_longest_position_holding_period = the_longest_position_holding_period.drop(['Last_FSCL_WK_END_DT_COM_TNUR', 'PRTNR_ACTIVE_FLG',
                                                                               'Last_FiscalWeekBeginDate',
                                                                               'Last_Month_Year','FiscalWeekBeginDate'], axis=1) 
the_longest_position_holding_period = the_longest_position_holding_period.rename(columns = {'FSCL_WK_END_DT_JOB_TNUR' : 'longest_position_holding_period'})


#### *generation group

In [2467]:
generation = state[['PRTNR_ID','GEN_GRP_CD']].drop_duplicates()

#### *Average count_of_prtnrs by Quarter and Branch to calculate labor density

In [2468]:
# Collect the cols 'PRTNR_ID','FiscalWeekBeginDate','PRTNR_ACTIVE_FLG','count_of_prtnrs' from the original dataset
count_of_prtnrs = state

#convert y-m-d to y-m
count_of_prtnrs['Month_Year']= pd.to_datetime(count_of_prtnrs['FiscalWeekBeginDate']).dt.to_period('M')
count_of_prtnrs['Qtr']= pd.to_datetime(count_of_prtnrs['FiscalWeekBeginDate']).dt.to_period('Q')

#Calculate Average Qtr by branch
Qtr_avg= round( pd.DataFrame(count_of_prtnrs.groupby(['Qtr','hashedstorenumber'])['count_of_prtnrs'].mean()).reset_index(),0)
Qtr_avg = Qtr_avg.rename(columns={'count_of_prtnrs' : 'Qtr_avg_count_of_prtnrs'})

#### If 'Count of partners on employee's active week' < 'Qtr_avg_count_of_prtnrs', 'Labor Density = 1'

In [2469]:
#Merge to count_of_prtnrs data-frame
count_of_prtnrs = pd.merge(count_of_prtnrs, Qtr_avg, how = 'left', left_on = ['Qtr','hashedstorenumber'],
                           right_on = ['Qtr','hashedstorenumber'])

count_of_prtnrs["labor density"] = [ 1 if x < y and z == 1 else 0 for x, y, z in zip(count_of_prtnrs["count_of_prtnrs"],
                                                                    count_of_prtnrs["Qtr_avg_count_of_prtnrs"],
                                                                                count_of_prtnrs["PRTNR_ACTIVE_FLG"])]

#### * labor density

In [2470]:
labor_density = pd.DataFrame(count_of_prtnrs.groupby(['PRTNR_ID'])['labor density'].sum()).reset_index()

#### * 'STATE_CODE','CITY_NAME', 'primary_venue', 'urbanity','store_type', 'hashedstorenumber' 

In [2471]:
# Collect the cols 'PRTNR_ID',STATE_CODE','CITY_NAME', 'primary_venue', 'urbanity','store_type', 'hashedstorenumber' from the original dataset
general = state[['PRTNR_ID','FiscalWeekBeginDate','STATE_CODE','CITY_NAME','urbanity','store_type', 'hashedstorenumber']].drop_duplicates()
general1 = general.groupby(['PRTNR_ID'])['FiscalWeekBeginDate'].max()
general2 = general[['PRTNR_ID','FiscalWeekBeginDate','STATE_CODE','CITY_NAME', 'urbanity','store_type', 'hashedstorenumber']]
general1 = pd.merge(general1,general2, how = 'left', left_on = ['PRTNR_ID','FiscalWeekBeginDate'] ,
                        right_on = ['PRTNR_ID','FiscalWeekBeginDate']).drop(['FiscalWeekBeginDate'], axis=1)

#### * benefitselig	& benefitsused

In [2472]:
# Collect the cols 'PRTNR_ID',STATE_CODE','CITY_NAME', 'primary_venue', 'urbanity','store_type',
benefit =state[['PRTNR_ID', 'FiscalWeekBeginDate', 'PRTNR_ACTIVE_FLG','benefitselig', 'benefitsused']] 
benefit = benefit.sort_values(by="FiscalWeekBeginDate")

#Find partners whose active flg is 0 and remain only the first 0
benefit_0 = benefit[benefit['PRTNR_ACTIVE_FLG']==0]
benefit_first_0 = pd.DataFrame(benefit_0.groupby(['PRTNR_ID'])['FiscalWeekBeginDate','PRTNR_ACTIVE_FLG','benefitselig','benefitsused'].agg('first')).reset_index()

benefit_second_0 = pd.merge(benefit_0, benefit_first_0, on=['PRTNR_ID','FiscalWeekBeginDate'],how='left', indicator=True).query("_merge=='left_only'").drop('_merge', 1)
benefit_second_0 = benefit_second_0.drop(['PRTNR_ACTIVE_FLG_y','benefitselig_y','benefitsused_y'], axis=1)
benefit_second_0 = benefit_second_0.rename(columns={'PRTNR_ACTIVE_FLG_x':'PRTNR_ACTIVE_FLG',
                                                   'benefitselig_x':'benefitselig',
                                                   'benefitsused_x':'benefitsused'})

benefit = pd.merge(benefit, benefit_second_0, on=['PRTNR_ID','FiscalWeekBeginDate'],how='left', indicator=True).query("_merge=='left_only'").drop('_merge', 1)
benefit = benefit.drop(['PRTNR_ACTIVE_FLG_y','benefitselig_y','benefitsused_y'], axis=1)
benefit = benefit.rename(columns={'PRTNR_ACTIVE_FLG_x':'PRTNR_ACTIVE_FLG',
                                                   'benefitselig_x':'benefitselig',
                                                   'benefitsused_x':'benefitsused'})

# Convert True and False to 1 and 0
benefit['benefitselig'].replace({False: 0, True: 1}, inplace=True)
benefit['benefitsused'].replace({False: 0, True: 1}, inplace=True)

# aggregate in a line for each 'PRTNR_ID'
benefitselig_sum= benefit.groupby(['PRTNR_ID'])['benefitselig'].sum().reset_index().round()
benefitsused_sum = benefit.groupby(['PRTNR_ID'])['benefitsused'].sum().reset_index()

<ipython-input-2472-97c3655ebd65>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  benefit_first_0 = pd.DataFrame(benefit_0.groupby(['PRTNR_ID'])['FiscalWeekBeginDate','PRTNR_ACTIVE_FLG','benefitselig','benefitsused'].agg('first')).reset_index()


#### *Obtaining pctpay

In [2473]:
state['FSCL_WK_END_DT_JOB_TNUR'] = round(state['FSCL_WK_END_DT_JOB_TNUR']/365,6)
pctpay_min = state[['PRTNR_ID','FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR' ,'pctpay','pulse']].groupby(['PRTNR_ID'])['FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR','pctpay','pulse'].min().reset_index().drop(['FiscalWeekBeginDate','pulse'], axis=1)
pctpay_max = state[['PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_JOB_TNUR','pctpay','pulse']].groupby(['PRTNR_ID'])['FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR' ,'pctpay','pulse'].max().reset_index().drop(['FiscalWeekBeginDate','pulse'], axis=1)

<ipython-input-2473-79a4f3ab1488>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pctpay_min = state[['PRTNR_ID','FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR' ,'pctpay','pulse']].groupby(['PRTNR_ID'])['FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR','pctpay','pulse'].min().reset_index().drop(['FiscalWeekBeginDate','pulse'], axis=1)
<ipython-input-2473-79a4f3ab1488>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pctpay_max = state[['PRTNR_ID','FiscalWeekBeginDate', 'FSCL_WK_END_DT_JOB_TNUR','pctpay','pulse']].groupby(['PRTNR_ID'])['FiscalWeekBeginDate','FSCL_WK_END_DT_JOB_TNUR' ,'pctpay','pulse'].max().reset_index().drop(['FiscalWeekBeginDate','pulse'], axis=1)


In [2474]:

pctpay_max = pctpay_max.rename(columns={'FSCL_WK_END_DT_JOB_TNUR':'MAX_FSCL_WK_END_DT_JOB_TNUR',
                                        'pctpay':'max_pctpay'})
pctpay_min = pctpay_min.rename(columns={'FSCL_WK_END_DT_JOB_TNUR':'MIN_FSCL_WK_END_DT_JOB_TNUR',
                                        'pctpay':'min_pctpay'})

pctpay_min_max = pctpay_min.merge(pctpay_max, how='left', on='PRTNR_ID')

pctpay_min_max['max-min_tnur'] = round((pctpay_min_max['MAX_FSCL_WK_END_DT_JOB_TNUR'] - pctpay_min_max['MIN_FSCL_WK_END_DT_JOB_TNUR'])/365,6)
pctpay_min_max['max-min_pctpay'] = pctpay_min_max['max_pctpay'] - pctpay_min_max['min_pctpay']

pctpay_min_max['anual_average_Wage_increase'] = round(pctpay_min_max['max-min_tnur']/pctpay_min_max['max-min_pctpay'],4)*100
pctpay_min_max = pctpay_min_max.drop(['MAX_FSCL_WK_END_DT_JOB_TNUR','max-min_tnur','max-min_pctpay'],axis =1)

#### *Create a new data-frame

In [2475]:
# # Merge the data-frame
new_df = earliest_FiscalWeekBeginDate.merge(the_number_of_jobs, how='inner', on='PRTNR_ID')
new_df = new_df.merge(last_position, how='inner', on='PRTNR_ID')
new_df = new_df.merge(FSCL_WK_END_DT_COM_TNUR, how='inner', on='PRTNR_ID')
new_df = new_df.merge(the_longest_position_holding_period, how='inner', on='PRTNR_ID')
new_df = new_df.merge(generation, how='inner', on='PRTNR_ID')
new_df = new_df.merge(labor_density, how='inner', on='PRTNR_ID')
new_df = new_df.merge(general1, how='inner', on='PRTNR_ID')
new_df = new_df.merge(benefitselig_sum, how='inner', on='PRTNR_ID')
new_df = new_df.merge(benefitsused_sum, how='inner', on='PRTNR_ID')
new_df = new_df.merge(pctpay_min_max, how='inner', on='PRTNR_ID').replace([np.inf, -np.inf], 0).dropna()

#### *Job Position Changed

In [2476]:
new_df["Job Position Changed"] = [ 1 if x > y else 0 for x, y in zip(new_df["Last_FSCL_WK_END_DT_COM_TNUR"],
                                                                                   new_df["longest_position_holding_period"])]

#### *Fromating Columns

In [2477]:
new_df['FiscalWeekBeginDate'] = pd.to_datetime(count_of_prtnrs['FiscalWeekBeginDate']).dt.to_period('M')
new_df['Last_FSCL_WK_END_DT_COM_TNUR'] = round(new_df['Last_FSCL_WK_END_DT_COM_TNUR']/365,2)
new_df['longest_position_holding_period'] = round(new_df['longest_position_holding_period']/365,2)
new_df[['MIN_FSCL_WK_END_DT_JOB_TNUR', 'min_pctpay','max_pctpay']] = round(new_df[['MIN_FSCL_WK_END_DT_JOB_TNUR', 'min_pctpay','max_pctpay']],2)
new_df = new_df.drop(['FiscalWeekBeginDate','count_position','Last_FiscalWeekBeginDate'], axis=1)
new_df

,PRTNR_ID,last_position,Last_FSCL_WK_END_DT_COM_TNUR,PRTNR_ACTIVE_FLG,Last_Month_Year,longest_position_holding_period,GEN_GRP_CD,labor density,STATE_CODE,CITY_NAME,urbanity,store_type,hashedstorenumber,benefitselig,benefitsused,MIN_FSCL_WK_END_DT_JOB_TNUR,min_pctpay,max_pctpay,anual_average_Wage_increase,Job Position Changed
0,0005F00557E5CD0BCAC6F8A1F343DC35,barista,1.34,1,2019-09,1.34,Generation Z,27,AZ,Mesa,U4,Drive Thru,24,71.0,1,0.02,0.41,0.44,10.33,0
1,000FF458D677F2E37B0EC7B3FB093968,shift supervisor,4.64,0,2019-04,3.21,Millennials,28,WA,Vancouver,U5,Drive Thru,4,82.0,0,1.67,0.70,0.78,4.98,1
2,0014B120A048068D92EE2A06B99E48A6,shift supervisor,9.46,0,2018-11,0.72,Millennials,20,NV,Las Vegas,U3,Drive Thru,264,61.0,0,0.02,0.46,0.55,7.12,1
4,001802D72E5685A8A81DAE17BE44ADD7,shift supervisor,3.24,0,2019-06,1.85,Generation Z,25,IL,Chicago,U2,Cafe,37,90.0,0,0.15,0.63,0.73,4.73,1
5,0018A858F59D9F3BFA5CF6AB93DA74B8,shift supervisor,0.78,1,2019-09,0.38,Millennials,13,WA,Tacoma,U4,Drive Thru,359,41.0,0,0.02,0.63,0.76,0.86,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13719,FFE432EE30DAE9BF78A45D3970ECF0A0,shift supervisor,0.87,0,2019-02,0.28,Generation Z,8,CA,Santa Clara,U3,Cafe,108,21.0,0,0.02,0.83,0.95,1.39,1
13720,FFF3F91FF8FFD54F8962C0C5DA8238BD,barista,0.16,0,2019-08,0.16,Generation Z,0,CA,San Diego,U3,Drive Thru,321,0.0,0,0.02,0.61,0.61,0.00,0
13721,FFF417C1B02B4DD879AD8B7FC4E8856A,barista,1.30,0,2018-07,1.30,Generation Z,6,OR,Portland,U4,Cafe,34,40.0,0,0.56,0.70,0.74,5.55,0
13722,FFF549C28AF0218448441980B3F4A52F,barista,0.60,0,2018-09,0.60,Generation Z,11,FL,Orlando,U5,Drive Thru,287,12.0,0,0.01,0.15,0.29,1.19,0


In [2478]:
list(new_df)

['PRTNR_ID',
 'last_position',
 'Last_FSCL_WK_END_DT_COM_TNUR',
 'PRTNR_ACTIVE_FLG',
 'Last_Month_Year',
 'longest_position_holding_period',
 'GEN_GRP_CD',
 'labor density',
 'STATE_CODE',
 'CITY_NAME',
 'urbanity',
 'store_type',
 'hashedstorenumber',
 'benefitselig',
 'benefitsused',
 'MIN_FSCL_WK_END_DT_JOB_TNUR',
 'min_pctpay',
 'max_pctpay',
 'anual_average_Wage_increase',
 'Job Position Changed']

#### *Re-ordering Columns

In [2479]:
new_df = new_df[['PRTNR_ID','Last_FSCL_WK_END_DT_COM_TNUR','MIN_FSCL_WK_END_DT_JOB_TNUR','longest_position_holding_period',
                 'last_position','Job Position Changed','labor density','PRTNR_ACTIVE_FLG', 'Last_Month_Year', 'GEN_GRP_CD',
                 'STATE_CODE','CITY_NAME','urbanity','store_type', 'hashedstorenumber', 'benefitselig','benefitsused',
                 'min_pctpay','max_pctpay','anual_average_Wage_increase'
                ]]

#### Change cols' name

In [2480]:
new_df = new_df.rename(columns = {'PRTNR_ID':'Prtnr_ID',
                                  'Last_FSCL_WK_END_DT_COM_TNUR': 'Tenure_Com',
                                  'MIN_FSCL_WK_END_DT_JOB_TNUR' :'Min_Tenure_Position',
                                  'longest_position_holding_period': 'Longest_Position',
                                  'last_position':'Last_Position',
                                  'Job Position Changed':'Job_changed',
                                  'labor density':'Labor_Density',
                                  'PRTNR_ACTIVE_FLG' :'Last_Active_Flg',
                                  'Last_Month_Year':'Last_Status_Month_Year',  
                                  'GEN_GRP_CD':'Gen',
                                  'STATE_CODE': 'State',
                                  'CITY_NAME' : 'City',
                                  'urbanity':'Urbanity',
                                  'store_type':'Store_Type',
                                  'hashedstorenumber':'Store_Num',
                                  'benefitselig':'Benefit_Selling',
                                  'benefitsused': 'Benefit_Used',
                                  'min_pctpay':'Min_Pctpay',
                                  'max_pctpay':'Max_Pctpay',
                                  'anual_average_Wage_increase':'Yearly_Wage_Increase(%)'
                                 })

#### *Get Dummies

In [2481]:
#Position dummies
position_dummy = pd.get_dummies(new_df['Last_Position'])
new_df= pd.concat([new_df,position_dummy], axis=1)

#GEN_GRP_CD dummies
gen_dummy = pd.get_dummies(new_df['Gen'])
new_df= pd.concat([new_df,gen_dummy], axis=1)

#state_code dummies
state_code_dummy = pd.get_dummies(new_df['State'])
new_df= pd.concat([new_df,state_code_dummy], axis=1)

#City dummies
city_dummy = pd.get_dummies(new_df['City'])
new_df= pd.concat([new_df,city_dummy], axis=1)

#Urbanity
u_dummy = pd.get_dummies(new_df['Urbanity'])
new_df = pd.concat([new_df,u_dummy], axis=1)

#Store_Type
store_type_dummy = pd.get_dummies(new_df['Store_Type'])
new_df= pd.concat([new_df,store_type_dummy], axis=1)
new_df

,Prtnr_ID,Tenure_Com,Min_Tenure_Position,Longest_Position,Last_Position,Job_changed,Labor_Density,Last_Active_Flg,Last_Status_Month_Year,Gen,State,City,Urbanity,Store_Type,Store_Num,Benefit_Selling,Benefit_Used,Min_Pctpay,Max_Pctpay,Yearly_Wage_Increase(%),assistant store manager,barista,cafe attendant,shift manager,shift supervisor,siren retail barista,siren retail operations lead,siren retail store manager,store manager,Baby Boomer,Generation X,Generation Z,Millennials,Pre-Baby Boomer,AL,AZ,CA,CO,DC,FL,GA,IL,IN,KS,KY,MA,NC,NM,NV,NY,...,Denver,El Paso,Everett,Fort Wayne,Fort Worth,Fredericksburg,Greenville,Houston,Indianapolis,Katy,Kent,La Mesa,Las Vegas,Long Beach,Los Angeles,Louisville,Marietta,Mesa,Miami,Mobile,Modesto,Naples,New York,Ontario,Orlando,Overland Park,Philadelphia,Portland,Richmond,Sacramento,San Diego,San Francisco,Santa Clara,Sarasota,Seattle,Spring,Tacoma,Tampa,Tempe,Tucson,Vancouver,Washington,U1,U2,U3,U4,U5,U6,Cafe,Drive Thru
0,0005F00557E5CD0BCAC6F8A1F343DC35,1.34,0.02,1.34,barista,0,27,1,2019-09,Generation Z,AZ,Mesa,U4,Drive Thru,24,71.0,1,0.41,0.44,10.33,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,000FF458D677F2E37B0EC7B3FB093968,4.64,1.67,3.21,shift supervisor,1,28,0,2019-04,Millennials,WA,Vancouver,U5,Drive Thru,4,82.0,0,0.70,0.78,4.98,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1
2,0014B120A048068D92EE2A06B99E48A6,9.46,0.02,0.72,shift supervisor,1,20,0,2018-11,Millennials,NV,Las Vegas,U3,Drive Thru,264,61.0,0,0.46,0.55,7.12,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
4,001802D72E5685A8A81DAE17BE44ADD7,3.24,0.15,1.85,shift supervisor,1,25,0,2019-06,Generation Z,IL,Chicago,U2,Cafe,37,90.0,0,0.63,0.73,4.73,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,0018A858F59D9F3BFA5CF6AB93DA74B8,0.78,0.02,0.38,shift supervisor,1,13,1,2019-09,Millennials,WA,Tacoma,U4,Drive Thru,359,41.0,0,0.63,0.76,0.86,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13719,FFE432EE30DAE9BF78A45D3970ECF0A0,0.87,0.02,0.28,shift supervisor,1,8,0,2019-02,Generation Z,CA,Santa Clara,U3,Cafe,108,21.0,0,0.83,0.95,1.39,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
13720,FFF3F91FF8FFD54F8962C0C5DA8238BD,0.16,0.02,0.16,barista,0,0,0,2019-08,Generation Z,CA,San Diego,U3,Drive Thru,321,0.0,0,0.61,0.61,0.00,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
13721,FFF417C1B02B4DD879AD8B7FC4E8856A,1.30,0.56,1.30,barista,0,6,0,2018-07,Generation Z,OR,Portland,U4,Cafe,34,40.0,0,0.70,0.74,5.55,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
13722,FFF549C28AF0218448441980B3F4A52F,0.60,0.01,0.60,barista,0,11,0,2018-09,Generation Z,FL,Orlando,U5,Drive Thru,287,12.0,0,0.15,0.29,1.19,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,

#### *drop employees who quit the job several times

In [2482]:
new_df =new_df.drop_duplicates()
duplicated = new_df[new_df['Prtnr_ID'].duplicated()]
duplicated = list(duplicated['Prtnr_ID'].drop_duplicates())
# pd.merge(a,b, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1))

In [4]:
to_drop = duplicated
new_df = new_df[~new_df['Prtnr_ID'].isin(to_drop)]

NameError: name 'duplicated' is not defined

##### Export Data File to CSV

In [2489]:
# # Export Data File to CSV
# new_df.to_csv("starbucks(12208)_FINAL.csv", index=None)

#### 12205 employees statistic summary

In [2485]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
new_df_describe = pd.DataFrame(new_df.describe())
new_df_describe

,Prtnr_ID,Tenure_Com,Min_Tenure_Position,Longest_Position,Last_Position,Job_changed,Labor_Density,Last_Active_Flg,Last_Status_Month_Year,Gen,State,City,Urbanity,Store_Type,Store_Num,Benefit_Selling,Benefit_Used,Min_Pctpay,Max_Pctpay,Yearly_Wage_Increase(%),assistant store manager,barista,cafe attendant,shift manager,shift supervisor,siren retail barista,siren retail operations lead,siren retail store manager,store manager,Baby Boomer,Generation X,Generation Z,Millennials,Pre-Baby Boomer,AL,AZ,CA,CO,DC,FL,GA,IL,IN,KS,KY,MA,NC,NM,NV,NY,...,Denver,El Paso,Everett,Fort Wayne,Fort Worth,Fredericksburg,Greenville,Houston,Indianapolis,Katy,Kent,La Mesa,Las Vegas,Long Beach,Los Angeles,Louisville,Marietta,Mesa,Miami,Mobile,Modesto,Naples,New York,Ontario,Orlando,Overland Park,Philadelphia,Portland,Richmond,Sacramento,San Diego,San Francisco,Santa Clara,Sarasota,Seattle,Spring,Tacoma,Tampa,Tempe,Tucson,Vancouver,Washington,U1,U2,U3,U4,U5,U6,Cafe,Drive Thru
12013,DFE0F3371C615110E29604F8303C04A4,9.08,3.53,4.95,shift supervisor,1,20,1,2019-09,Millennials,CA,Modesto,U4,Drive Thru,254,76.0,1,0.50,0.65,2.64,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4733,578A9DEDEABC80F9C4BA7C55ABC1223F,0.42,0.02,0.42,barista,0,10,1,2019-09,Generation Z,NC,Charlotte,U5,Cafe,420,0.0,0,0.02,0.02,0.00,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
7976,94AA83CE9EEEA2C85358F6E1DF0A1516,4.10,2.13,4.10,barista,0,61,1,2019-09,Generation Z,CA,Bakersfield,U5,Drive Thru,428,142.0,38,0.46,0.72,2.09,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
12700,EC6B20C13F5A87F9DB00AAB25D5BF9B6,3.09,0.92,2.22,shift supervisor,1,18,1,2019-09,Generation Z,PA,Philadelphia,U2,Cafe,296,69.0,0,0.57,0.68,3.20,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2035,25A29EA3D401DBF9DD209790B1155A2E,0.44,0.02,0.44,barista,0,5,0,2018-11,Millennials,TX,Spring,U5,Drive Thru,225,23.0,0,0.15,0.15,0.00,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8164,985CEA2F375C165FB8B7F64E6859E4A8,4.75,1.76,3.39,shift supervisor,1,35,0,2019-05,Generation Z,NY,Brooklyn,U2,Cafe,337,86.0,0,0.93,0.97,10.76,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5333,62AFC770AA2956C051E4A98112F73BB4,3.42,1.45,3.42,barista,0,41,1,2019-09,Millennials,AZ,Tucson,U5,Drive Thru,439,39.0,1,0.41,0.44,15.43,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
9776,B701EC90F5508BEC98C07EA22914D713,0.43,0.02,0.43,barista,0,6,0,2019-09,Baby Boomer,WA,Seattle,U4,Cafe,395,3.0,0,0.99,0.99,0.00,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
10759,C8C01A45BCF287A62C9AB3105E114E8B,0.17,0.02,0.17,barista,0,10,1,2019-09,Generation Z,SC,Greenville,U5,Drive Thru,275,0.0,8,0.06,0.06,0.00,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
new_df_median = pd.DataFrame(new_df.median()).transpose().rename(index={0: 'median'})
new_df_median

#### Obtain talented employees

In [ ]:
new_df_talented = new_df[(new_df['Tenure_Com']> 1.09) ]
new_df_talented = new_df[(new_df['Longest_Position']> 0.83) ]
new_df_talented.head()

In [ ]:
# new_df_talented.to_csv('talented_employee.csv')